In [7]:
import csv
import json
import os 
import string

import pandas as pd
import nltk
import googletrans
import langid

os.system('python -m nltk.downloader stopwords')

0

In [8]:
df = pd.read_csv('nlp_out.csv', header = 0)

#convert json string columns to json
df[df.filter(like="_j").columns] = df.filter(like="_j").applymap(lambda x : json.loads(x))

#change column names
for v in df.filter(like="_j"):
    df = df.rename(columns = {v : '_'.join(v.split('_')[:-1])})

In [5]:
df

,onedim,twodim,reliability,severity,demo_groups,specific_needs,aff_groups,geo,info_date,excerpt,has_image,lead_text,lead_id,lead_url,event
0,{'Population Profile': ['Demographic Profile']},None,Usually,Situation of Concern,[Children (5 to 11 years old)],None,[Affected],None,2016-11-15,"894,057 children are among the 2.1 million aff...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
1,None,"[{'sector': 'Nutrition', 'subsectors': ['Breas...",Usually,Severe Conditions,[Infants/toddlers (< 5 years old)],None,[Affected],None,2016-11-15,"112,500 children under five are at risk of acu...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
2,None,"[{'sector': 'Health', 'subsectors': ['Health s...",Usually,Severe Conditions,None,None,[Affected],None,2016-11-15,"2,271 cholera cases suspected between 4-19 Oct...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
3,None,"[{'sector': 'Food', 'subsectors': None, 'pilla...",Usually,Severe Conditions,None,None,[Affected],None,2016-11-15,"806,000 people need urgent food assistance.",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
4,None,"[{'sector': 'Health', 'subsectors': None, 'pil...",Usually,Situation of Concern,None,None,[Affected],None,2016-11-15,36 health facilities destroyed.,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
5,{'Population Profile': ['Humanitarian Profile']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,2.1 million Affected people,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
6,None,"[{'sector': 'Cross', 'subsectors': None, 'pill...",Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,1.4 million People need aid \n,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
7,{'Context': ['Overview']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,"On 4 October, Hurricane Matthew violently stru...",True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
8,{'Population Profile': ['Humanitarian Profile']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,The latest figures from the governmental Direc...,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event
9,{'Humanitarian access': ['Physical Constraints']},None,Usually,Situation of Major Concern,None,None,[Affected],None,2016-11-15,Though access continues to be gained to more a...,True,None,4,http://reliefweb.int/sites/reliefweb.int/files...,Test Event


In [3]:
#approx check types of a row to ensure json conversion
for v in df.iloc[0]:
    print(type(v))

<class 'dict'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'list'>
<class 'NoneType'>
<class 'list'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'numpy.bool_'>
<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>


In [18]:
#flatten dict columns

df

[{'pillar': 'Capacities and Response',
  'sector': 'Food',
  'subpillar': 'International Response',
  'subsectors': None}]

In [23]:
get_sub_sectors_excerpt()

[(['Nutrition', ['Breastfeeding']],
  '112,500 children under five are at risk of acute malnutrition.'),
 (['Health', ['Health status and risk']],
  '2,271 cholera cases suspected between 4-19 October alone. '),
 (['Food', None], '806,000 people need urgent food assistance. '),
 (['Health', None], '36 health facilities destroyed. '),
 (['Cross', None], '1.4 million People need aid \n'),
 (['Cross', None],
  'Humanitarian needs are said to include access to a sufficient supply of quality water, education, shelter, child protection, health, and nutrition. '),
 (['Food', None],
  'The people in urgent food insecurity are located in areas where over 75 per cent of the population was affected by the hurricane. These include places where livelihood activities related to agriculture, livestock and fishing have been almost completely destroyed, such as crops, farming equipment, stocks, and trade. '),
 (['Agriculture', None],
  'The people in urgent food insecurity are located in areas where ov

In [19]:
#help functions
def get_all_sectors():
    """return raw list of sectors"""
    l = []
    for v in df[df['twodim'].notnull()][['twodim']].iterrows():
        for k in v[1].twodim:
            l.append(k['sector'])
    
    return l
    
def get_sector_excerpt():
    """return list of tuples with sector and excerpt"""
    l = []

    for v in df[df['twodim'].notnull()][['twodim','excerpt']].iterrows():
        for k in v[1].twodim:
            l.append((k['sector'], v[1].excerpt))
    
    return l

def get_sub_sectors_excerpt():
    """return list of tuples with sector, subsectors and excerpt"""
    l = []

    for v in df[df['twodim'].notnull()][['twodim','excerpt']].iterrows():
        for k in v[1].twodim:
            l.append(([k['sector'], k['subsectors']], v[1].excerpt))
    
    return l

In [6]:
##numerical checks##

#groups of sectors, should match:

"""
select name, c from
(
    select sector_id, count(*) as c from entries_informationattribute i
    group by 1
)a 
join entries_sector e
on a.sector_id = e.id
"""
l = get_all_sectors()
pd.Series(l).groupby(l).size()

Agriculture     919
Cross          1037
Education      1218
Food           4813
Health         5148
Livelihood     1843
Logistic        539
NFI            1282
Nutrition      1288
Protection     5020
Shelter        2641
WASH           2244
dtype: int64

In [ ]:
"2.1 million"

In [14]:
def rm_punc_not_nums(inp, col):
    """remove punctuation unless it's a number for either a df (and col) or single entry"""
    punc =  string.punctuation
    transtable = str.maketrans("","", punc)
    
    def sing_rm(phr):
        """remove for a single entity"""
        return ' '.join ([i.translate(transtable) if not (all(j.isdigit() or j in punc for j in i) 
                    and any(j.isdigit() for j in i)) else i for i in str(phr).split(' ')])
        
    if isinstance(inp, pd.core.frame.DataFrame):
        return inp.filter(like=col).applymap(lambda phr : sing_rm(phr))
    
    elif isinstance(inp, str):
        return sing_rm(inp)
                            
    else:
        raise Exception('Not a vaild type')

def rm_stop_words(df, col, swords = nltk.corpus.stopwords.words('english')):
    """remove stop words from a given column in a dataframe"""
    return df.filter(like=col).applymap(lambda ent : 
                    ' '.join([token for token in str(ent).split(' ') if token.lower() not in swords]))

In [15]:
##clean excerpts##

#TODO: get better stop words, remove relevant words (ie under for under 5)
EXCERPT_COL = 'excerpt'

#remove punctuation
df[EXCERPT_COL] = rm_punc_not_nums(df, EXCERPT_COL)

#stop words
df[EXCERPT_COL] = rm_stop_words(df, EXCERPT_COL)                                                   

#check lang distribution
lang = pd.Series([langid.classify(str(v))[0] for v in df[EXCERPT_COL]])
lang.groupby(lang).size()

NameError: name 'langid' is not defined

In [26]:
langid.classify('hola como estas')

('es', -57.14961767196655)

In [17]:
lang = pd.Series([langid.classify(str(v))[0] for v in df[EXCERPT_COL]])
lang.groupby(lang).size()

af       19
am        2
an       11
az        1
br       16
bs        1
ca       32
cs        1
cy        3
da       95
de      204
en    28983
eo       21
es      736
et       45
eu       22
fi        6
fo        2
fr     3765
ga        2
gl        2
he        1
hr        4
ht       24
hu        4
id      158
is        2
it      126
ja        1
jv       16
la       39
lb        1
lt       10
lv        8
mg        8
ms       30
mt       14
nl       95
nn        1
no       38
oc        9
pl       16
pt       55
qu        4
ro        8
rw       81
se        5
sl       15
sv       30
sw       55
tl       68
tr        3
vi        2
vo        1
wa        2
xh        6
zh        1
zu        7
dtype: int64